In [ ]:
from tkinter import *
import tkinter as tk
from tkinter import *
from PIL import ImageTk, Image
from time import strftime
import datetime as dt
import tkinter as tk
from tkinter import filedialog
from tkinter.filedialog import askopenfile
from PIL import Image, ImageTk
from tkinter import filedialog
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import os
import pandas as pd
import random
import csv
# import xlwt
# from xlwt import Workbook
import matplotlib.pyplot as plt
from tkinter.scrolledtext import ScrolledText

from tkinter import *
import tkinter as tk
from tkinter import ttk  # Add this line
from PIL import ImageTk, Image
from time import strftime
import datetime as dt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import load_model
import pandas as pd
import seaborn as sns
from tkinter import messagebox
from tkinter.filedialog import askopenfilename
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Define global variables
model = None
X_test = None
Y_test = None
predictions = None
scaler_input = None
scaler_output = None
predictions = None
history = None
z_p = None



def train_model():
    global model, X_test, Y_test, scaler_input, scaler_output
    try:
        # Load the pre-trained model
        model = load_model('trained_model.h5')
        # Load the dataset
        dataset = pd.read_csv("C:/Users/hp/OneDrive/Desktop/oldram.csv")
        # Selecting columns for input (X) and output (y)
        X = dataset[['Velocity', 'solid', 'Time', '    x-coordinate', '    y-coordinate', '    z-coordinate']]
        Y = dataset[['water-mean-z-velocity', '    air-mean-vof', '   mean-pressure', '  solid-mean-vof']]
        # Scaling the input features using StandardScaler
        scaler_input = StandardScaler()
        X_scaled = scaler_input.fit_transform(X)
        # Scaling the output features using MinMaxScaler
        scaler_output = MinMaxScaler()
        Y_scaled = scaler_output.fit_transform(Y)
        # Splitting the scaled data into training and testing sets
        X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_scaled, test_size=0.2, random_state=42)
        messagebox.showinfo("Model Training", "Model trained successfully!")
    except Exception as e:
        messagebox.showerror("Error", str(e))

def predict():
    global predictions
    try:
        if model is None or X_test is None or Y_test is None or scaler_output is None:
            raise Exception("Model, X_test, Y_test, or scaler_output is not defined. Please train the model first.")
        # Make predictions
        predictions = model.predict(X_test)
        messagebox.showinfo("Prediction", "Predictions made successfully!")
    except Exception as e:
        messagebox.showerror("Error", str(e))

def plot_data(x, y, xlabel, ylabel):
    plt.figure(figsize=(6, 4))
    plt.plot(x, y)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.tight_layout()
    return plt

def generate_data_axial(parameter, velocity, solid):
    if scaler_input is None:
        messagebox.showerror("Error", "Scaler input is not initialized. Please train the model first.")
        return
    z_p = pd.DataFrame()
    for k in np.linspace(0, 0.9, 100):
        pre = list(scaler_output.inverse_transform(model.predict(scaler_input.transform([[velocity, solid, 15, 0, 0, k]]))))
        z_p = z_p.append(pre)
    z_p.columns = ['Liquid velocity(m/s)', 'Gas holdup(-)', 'Static pressure(Pa)', 'Solid holdup(-)']
    y = z_p[parameter]
    x = np.linspace(0, 0.9, 100)
    return x, y

def plot_on_gui_axial():
    velocity = float(velocity_axial_entry.get())
    solid = float(solid_axial_entry.get())
    parameter = parameter_selection.get()

    x, y = generate_data_axial(parameter, velocity, solid)
    plot_data(x, y, 'Axial location (m)', parameter)

    plt.savefig("temp_axial_plot.png")  # Save the plot to a temporary file

    # Clear the existing plot if any
    for widget in frame_plot.winfo_children():
        widget.destroy()

    # Display the plot within the GUI
    image = Image.open("temp_axial_plot.png")
    photo = ImageTk.PhotoImage(image)
    label = tk.Label(frame_plot, image=photo)
    label.image = photo
    label.pack(fill=tk.BOTH, expand=True)


def generate_data_radial(model, scaler_input, scaler_output, velocity, solid, axial, selected_parameter, direction):
    z_p = pd.DataFrame()
    if direction == 'X':
        for i in np.linspace(-0.09972038, 0.09972038, 40):
            pre = list(scaler_output.inverse_transform(model.predict(scaler_input.transform([[velocity, solid, 15, i, 0, axial]]))))
            z_p = z_p.append(pre)
    elif direction == 'Y':
        for j in np.linspace(-0.09972038, 0.09972038, 40):
            pre = list(scaler_output.inverse_transform(model.predict(scaler_input.transform([[velocity, solid, 15, 0, j, axial]]))))
            z_p = z_p.append(pre)
    z_p.columns = ['Liquid velocity(m/s)', 'Gas holdup(-)', 'Static pressure(Pa)', 'Solid holdup(-)']
    return np.linspace(-0.09972038, 0.09972038, 40), z_p[selected_parameter]

def plot_radial(x, y, xlabel, ylabel):
    plt.figure(figsize=(6, 4))
    plt.plot(x, y)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.tight_layout()
    plt.grid(True)
    return plt

def predict_and_plot_radial():
    velocity_str = velocity_radial_entry.get()
    solid_str = solid_radial_entry.get()
    axial_str = axial_radial_entry.get()

    if not velocity_str or not solid_str or not axial_str:
        messagebox.showerror("Error", "Please enter the values of velocity, solid, and axial.")
        return

    try:
        velocity = float(velocity_str)
        solid = float(solid_str)
        axial = float(axial_str)
    except ValueError:
        messagebox.showerror("Error", "Invalid input. Please enter valid numeric values for velocity, solid, and axial.")
        return

    selected_parameter = parameter_radial_var.get()
    direction = direction_radial_var.get()

    x, y = generate_data_radial(model, scaler_input, scaler_output, velocity, solid, axial, selected_parameter, direction)
    plot_radial(x, y, 'Width of Bed (m)', selected_parameter)
    
    plt.savefig("temp_radial_plot.png")  # Save the plot to a temporary file

    # Clear the existing plot if any
    for widget in frame_plot.winfo_children():
        widget.destroy()

    # Display the plot within the GUI
    image = Image.open("temp_radial_plot.png")
    photo = ImageTk.PhotoImage(image)
    label = tk.Label(frame_plot, image=photo)
    label.image = photo
    label.pack(fill=tk.BOTH, expand=True)




# GUI initialization
root = tk.Tk()
root.title("DNN Model GUI")
root.geometry("1000x800")
root.configure(bg='honeydew2')

# Create notebook (tabbed interface)
notebook = ttk.Notebook(root)
notebook.pack(fill=tk.BOTH, expand=True)

# Training tab
frame_train = ttk.Frame(notebook)  
notebook.add(frame_train, text='Model Training')

# Plotting frame
frame_plot = ttk.Frame(root, width=500, height=400)
frame_plot.pack_propagate(False)
frame_plot.place(x=480, y=150)

# Graph plot placeholder
canvas = tk.Canvas(frame_plot, bg='white', width=400, height=400)
canvas.pack(fill=tk.BOTH, expand=True)

# File path entry and browse button
lbl_file_path = ttk.Label(frame_train, text="Select CSV File:", font=('calibri', 15, 'bold'), foreground='blue')
lbl_file_path.place(x=10, y=10)



# Train model button
btn_train_model = ttk.Button(frame_train, text="Load Model", command=train_model)
btn_train_model.place(x=10, y=70)

# Predict button
btn_predict = ttk.Button(frame_train, text="Predict", command=predict)
btn_predict.place(x=150, y=70)

# Create parameter selection dropdown
parameter_axial_label = ttk.Label(frame_train, text="Hydrodynamic Parameter:",  font=('calibri', 15, 'bold'), foreground='blue')
parameter_axial_label.place(x=10, y=100)
parameters = ['Liquid velocity(m/s)', 'Gas holdup(-)', 'Static pressure(Pa)', 'Solid holdup(-)']
parameter_selection = tk.StringVar(frame_train)
parameter_selection.set(parameters[0])  # default value
parameter_dropdown = tk.OptionMenu(frame_train, parameter_selection, *parameters)
parameter_dropdown.place(x=250, y=100)

# Velocity axial
velocity_axial_label = ttk.Label(frame_train, text="Velocity (cm/s):",  font=('calibri', 15, 'bold'), foreground='purple')
velocity_axial_label.place(x=10, y=150)
velocity_axial_entry = ttk.Entry(frame_train)
velocity_axial_entry.place(x=250, y=150)

# Solid loading axial
solid_axial_label = ttk.Label(frame_train, text="Solid Loading (%):", font=('calibri', 15, 'bold'), foreground='purple')
solid_axial_label.place(x=10, y=180)
solid_axial_entry = ttk.Entry(frame_train)
solid_axial_entry.place(x=250, y=180)

# Plot Axial button
plot_button_axial = ttk.Button(frame_train, text="Plot Axial", command=plot_on_gui_axial)
plot_button_axial.place(x=250, y=210)

# Velocity radial
velocity_radial_label = ttk.Label(frame_train, text="Velocity (cm/s):", font=('calibri', 15, 'bold'), foreground='purple')
velocity_radial_label.place(x=10, y=240)
velocity_radial_entry = ttk.Entry(frame_train)
velocity_radial_entry.place(x=250, y=240)

# Solid radial
solid_radial_label = ttk.Label(frame_train, text="Solid Loading (%):", font=('calibri', 15, 'bold'), foreground='purple')
solid_radial_label.place(x=10, y=270)
solid_radial_entry = ttk.Entry(frame_train)
solid_radial_entry.place(x=250, y=270)

# Axial radial
axial_radial_label = ttk.Label(frame_train, text="Axial Location (m):", font=('calibri', 15, 'bold'), foreground='purple')
axial_radial_label.place(x=10, y=300)
axial_radial_entry = ttk.Entry(frame_train)
axial_radial_entry.place(x=250, y=300)

# Direction radial dropdown
direction_radial_label = ttk.Label(frame_train, text="Radial Direction:", font=('calibri', 15, 'bold'), foreground='purple')
direction_radial_label.place(x=10, y=340)
direction_radial = ['X', 'X', 'Y']
direction_radial_var = tk.StringVar(frame_train)
direction_radial_var.set(direction_radial[0])
direction_radial_dropdown = ttk.OptionMenu(frame_train, direction_radial_var, *direction_radial)
direction_radial_dropdown.place(x=250, y=340)



parameter_radial_label = ttk.Label(frame_train, text="Hydrodynamic Parameter:",  font=('calibri', 15, 'bold'), foreground='blue')
parameter_radial_label.place(x=10, y=380)
parameters_radial = ['Liquid velocity(m/s)', 'Gas holdup(-)', 'Static pressure(Pa)', 'Solid holdup(-)']
parameter_radial_var = tk.StringVar(frame_train)
parameter_radial_var.set(parameters[0])  # default value
parameter_radial_dropdown = tk.OptionMenu(frame_train, parameter_radial_var, *parameters_radial)
parameter_radial_dropdown.place(x=250, y=380)

# Plot Radial button
plot_button_radial = ttk.Button(frame_train, text="Plot Radial", command=predict_and_plot_radial)
plot_button_radial.place(x=250, y=420)

# Time Key
def time():
    string = strftime('%H:%M:%S %p')
    mark.config(text=string)
    mark.after(1000, time)

mark = ttk.Label(frame_train, font=('calibri', 20, 'bold'), foreground='purple')
mark.place(x=600, y=550)
time()

# Date Key
date = dt.datetime.now()
# Create Label to display the Date
label = ttk.Label(frame_train, text=f"{date:%A, %B %d, %Y}", font=('calibri', 20, 'bold'), foreground='purple')
label.place(x=600, y=580)

btn = ttk.Button(frame_train, text="ML-CFD GUI", style="Accent.TButton")
btn.place(x=0, y=550)

style = ttk.Style()
style.configure("Accent.TButton", foreground="red", background="white", font=("Helvetica", 30, "bold"))

# Load and display the logo image
image_logo = Image.open("logo.png")
image_logo_resized = image_logo.resize((250, 100))
photo_logo = ImageTk.PhotoImage(image_logo_resized)
label_logo = tk.Label(frame_train, image=photo_logo)
label_logo.place(x=700, y=0)




# Heading label for notes
notes_heading_label = ttk.Label(frame_train, text="Note: Please pay attention to the range of input parameters", font=('calibri', 12, 'bold'), foreground='red')
notes_heading_label.place(x=10, y=620)

# Note about the range of input parameters for velocity
velocity_note_label = ttk.Label(frame_train, text="Velocity Range:", font=('calibri', 10, 'bold'), foreground='blue')
velocity_note_label.place(x=10, y=640)

velocity_value_label = ttk.Label(frame_train, text="10 cm/s to 20 cm/s", font=('calibri', 10, 'bold'), foreground='green')
velocity_value_label.place(x=130, y=640)


# Note about the range of input parameters for solid loading
solid_note_label = ttk.Label(frame_train, text="Solid Loading Range:", font=('calibri', 10, 'bold'), foreground='blue')
solid_note_label.place(x=10, y=660)

solid_value_label = ttk.Label(frame_train, text="5 % to 10 %", font=('calibri', 10, 'bold'), foreground='green')
solid_value_label.place(x=130, y=660)


# Note about the range of input parameters for axial location
axial_note_label = ttk.Label(frame_train, text="Axial Location Range:", font=('calibri', 10, 'bold'), foreground='blue')
axial_note_label.place(x=10, y=680)

axial_value_label = ttk.Label(frame_train, text="0 m to 0.9 m", font=('calibri', 10, 'bold'), foreground='green')
axial_value_label.place(x=130, y=680)





root.mainloop()


C:\Users\hp\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"




1/1 [==============================] - 0s 69ms/step


C:\Users\hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\hp\AppData\Local\Temp\ipykernel_1208\2636159304.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  z_p = z_p.append(pre)
C:\Users\hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 21ms/step


C:\Users\hp\AppData\Local\Temp\ipykernel_1208\2636159304.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  z_p = z_p.append(pre)
C:\Users\hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\hp\AppData\Local\Temp\ipykernel_1208\2636159304.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  z_p = z_p.append(pre)
C:\Users\hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\hp\AppData\Local\Temp\ipykernel_1208\2636159304.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  z_

...

C:\Users\hp\AppData\Local\Temp\ipykernel_1208\2636159304.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  z_p = z_p.append(pre)
C:\Users\hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\hp\AppData\Local\Temp\ipykernel_1208\2636159304.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  z_p = z_p.append(pre)
C:\Users\hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\hp\AppData\Local\Temp\ipykernel_1208\2636159304.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  z_

1/1 [==============================] - 0s 10ms/step


C:\Users\hp\AppData\Local\Temp\ipykernel_1208\2636159304.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  z_p = z_p.append(pre)
C:\Users\hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\hp\AppData\Local\Temp\ipykernel_1208\2636159304.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  z_p = z_p.append(pre)
C:\Users\hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\hp\AppData\Local\Temp\ipykernel_1208\2636159304.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  z_

1/1 [==============================] - 0s 10ms/step


C:\Users\hp\AppData\Local\Temp\ipykernel_1208\2636159304.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  z_p = z_p.append(pre)
C:\Users\hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\hp\AppData\Local\Temp\ipykernel_1208\2636159304.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  z_p = z_p.append(pre)
